<a href="https://colab.research.google.com/github/BerkeYrcr/Preparing-dataset-with-roboflow-for-yolov5/blob/main/yolo_Train_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Yolov5 Custom dataset eğitimi**

# **Kurulum**

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # yolov5'i klonluyoruz
%cd yolov5 #yolov5 klasörüne giriş yapıyoruz
%pip install -qr requirements.txt # gerekli kütüphaneleri indiriyoruz
%pip install -q roboflow #roboflow'u indiriyoruz
%pip install nvidia-tensorrt#tensorrt'yi indiriyoruz
import torch
import os
from IPython.display import Image, clear_output  # resimleri görüntülemek için kütüphaneyi import ediyoruz

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})") #hangi ekran kartının kullanıldığını yazdırıyoruz

# **Roboflowu aktif ediyoruz ve roboflowdaki verisetimizi indiriyoruz**

In [3]:
from roboflow import Roboflow 
rf = Roboflow(model_format="yolov5", notebook="ultralytics") 
# environment'ı kuruyoruz
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
#roboflowdan aldığımız api kodlarını kopyalıyoruz çalıştırınca indirecek
rf = Roboflow(api_key="7vXEPUGzaKswGlwleHmN")
project = rf.workspace("savasaniha").project("droness-wk9ba")
dataset = project.version(3).download("yolov5")

# **Modelin eğitimine başlıyoruz**

In [ ]:
# 640*640'lık resimleri 8 lik batchler halinde 100 epochda eğitiyoruz.Bu kısım elinizdeki datasetin büyüklüğüne göre değişebilir ve saatler sürebilir.
!python train.py --img 640 --batch 8 --epochs 100 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

# **Modelin başarı oranını grafikller halinde bakıyoruz**

In [ ]:
# tensorboard'ı aktif ediyoruz
# runs klasörünün içine kaydediyoruz
%load_ext tensorboard
%tensorboard --logdir runs

# **Test verilerini deniyoruz**

In [ ]:
#test resimlerimizi işliyoruz ve elde edilen sonuçları bir sonraki adımda görüyoruz.
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 416 --conf 0.2 --source {dataset.location}/valid/images

# **Test verileri ekranda gösteriyoruz**

In [ ]:

#İşlediğimiz verileri ekranda görünmesini sağlıyoruz
import glob
from IPython.display import Image, display

i = 0
# Dosya uzantısının doğru olduğundan emin olunuz eğer birdan fazla test yaptıysanız exp,exp2 gibi klasörler oluşacaktır dosya yolundan kontrol edin
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #dosya uzantısını kontrol ediniz jpg olmak zorunda değil
    i += 1

    if i < 12:
      display(Image(filename=imageName))
      print("\n")

# **Modeli kaydediyoruz**

In [ ]:
#modelimizi kaydediyoruz.Eğer bilgisayarınıza indirmek isterseniz aşağıda yazan dosya yolundan indirebilirsiniz.
from google.colab import files
files.download('/content/yolov5/runs/train')

# **Dataseti tensorrtye değiştirme için gerekli kütüphaneleri yüklüyoruz**


In [ ]:
!pip install -r /content/yolov5/requirements.txt coremltools onnx onnx-simplifier onnxruntime-gpu openvino-dev tensorflow

# **Pytorch Datasetini Tensorrt datasetine çevirme**


In [ ]:
!python export.py --device 0 --weights /content/yolov5/yolov5s.pt --include engine

# **Yaptığımız dataseti benchmarka sokuyoruz(**ZORUNLU DEĞİL**)**

In [ ]:
!python benchmarks.py --weights yolov5s.pt --img 640